# Enhancer Prediction

This notebook is an attempt at predicting enhancer regions based on open chromatin/1D omic data. It is simlar to the [Anshul Kundaje lab's](https://simons.berkeley.edu/sites/default/files/docs/4575/2016-kundaje-simonsinstitute-deeplearning.pdf) attempts at predicting chromatin state/other chromatin marks based on ATAC-seq, Dnase-seq, and MNase-seq data. Data used in this repository comes from Roadmap's processed data for [Dnase/Histone ChIP](http://egg2.wustl.edu/roadmap/web_portal/processed_data.html) and for [ChromHMM genomic annotations](http://egg2.wustl.edu/roadmap/web_portal/chr_state_learning.html#exp_18state).  

For this example, I will use Roadmap Epiginome E116, which is the cell line GM12878. 

## Data Processing

To start, we need to download the Dnase-seq ([peaks](http://egg2.wustl.edu/roadmap/data/byFileType/peaks/consolidated/narrowPeak/) + [tagAlign](http://egg2.wustl.edu/roadmap/data/byFileType/alignments/consolidated) files), and [ChromHMM annotations](http://egg2.wustl.edu/roadmap/web_portal/chr_state_learning.html#exp_18state) from Roadmap. I will use the expanded 18 state annotation because it has H3K27ac and contains more states. All of the starting data except the tagAlign file are available in the data folder. 

First lets see how many peaks/states we have:
```
> wc -l E116-DNase.macs2.narrowPeak
271214 E116-DNase.macs2.narrowPeak
> wc -l E116_18_core_K27ac_dense.bed
569183 E116_18_core_K27ac_dense.bed
```

There are ~270k Dnase paeks and ~570k chromatin states. Then I will to extract the enhancer regions from the ChromHMM annotations. This is what the head of the annotation for E116 looks like:

```
track name="E116_18_core_K27ac" description=" E116_18_core_K27ac (Emission ordered)" visibility=1 itemRgb="On"
chr10   0       119600  18_Quies        0       .       0       119600  255,255,255
chr10   119600  120200  2_TssFlnk       0       .       119600  120200  255,69,0
chr10   120200  136000  18_Quies        0       .       120200  136000  255,255,255
chr10   136000  149400  13_Het  0       .       136000  149400  138,145,208
chr10   149400  151400  12_ZNF/Rpts     0       .       149400  151400  102,205,170
chr10   151400  153200  13_Het  0       .       151400  153200  138,145,208
chr10   153200  156800  18_Quies        0       .       153200  156800  255,255,255
chr10   156800  159800  17_ReprPCWk     0       .       156800  159800  192,192,192
chr10   174200  174400  11_EnhWk        0       .       174200  174400  255,255,0
chr10   174400  175600  6_TxWk  0       .       174400  175600  0,100,0
chr10   175600  176000  11_EnhWk        0       .       175600  176000  255,255,0
```

Column 4 has the different states. I want all state that have "Enh" for enhancers:

```
awk '$4 ~ "Enh" {print $0}' E116_18_core_K27ac_dense.bed > all_enh.bed
```

```
> wc -l all_enh.bed
160714 all_enh.bed
```
160k out of the 570k states are enhancers (~30% of all states). I will then use bedtools to find which Dnase peaks are in enhancer regions:

```
>bedtools intersect -wa -u -a E116-DNase.macs2.narrowPeak -b all_enh.bed > Dnase_enhancers.bed
>bedtools intersect -v -a E116-DNase.macs2.narrowPeak -b all_enh.bed > Dnase_non_enhancers.bed
```

```
> wc -l Dnase_enhancers.bed 
95033 Dnase_enhancers.bed
```

```
> wc -l Dnase_non_enhancers.bed
176181 Dnase_non_enhancers.bed
```

From the Dnase data, 95k out of the 270k peaks (35%) contain enhancer regions.

Next, I will use the script `change_peak_lengths.py` to extend or shrink all peaks to 2 kb:
```
usage: change_peak_lengths.py [-h] [--fix] [--extend] peak dist

Change peak widths. By default, this will extend peaks to specified length and
merge overlapping peaks

positional arguments:
  peak        Peak file to modify
  dist        Distance to extend peaks. Default is 1kb. If --fix is not set,
              peaks shorter than dist will be extended to dist.

optional arguments:
  -h, --help  show this help message and exit
  --fix       Indicate if you want to fix peak widths. This will shorten peaks
              longer than dist to be equal to dist. If --extend is not set,
              this will merge peaks first and then fix peaks to specified size
  --extend    Indicate if you want to extend peaks only and not merge
```

Now to run the script:

```
>python change_peak_lengths.py Dnase_enhancers.bed 2000 --fix --extend
>python change_peak_lengths.py Dnase_non_enhancers.bed 2000 --fix --extend
```


The outputs will be labeled as `Dnase_enhancers_2000bp_extend_fixed.bed` for enhancers and `Dnase_non_enhancers_2000bp_extend_fixed.bed` for the non enhancer regions.  

Then I will only select non-enhancers that do not overlap any enhancers. When extending the peaks by 2kb, I might extended some non-enhancer regions into enhancer regions. If a non-enhancer overlapping Dnase peak is within 2kb of an enhancer Dnase peak, then for the purposes of this analysis, it will be considered to be a part of the enhancer peak:

```
>bedtools intersect -v -a Dnase_non_enhancers_2000bp_extend_fixed.bed -b Dnase_enhancers_2000bp_extend_fixed.bed > Dnase_non_enhancers_2000bp_extend_fixed_filtered.bed
```

```
> wc -l Dnase_non_enhancers_2000bp_extend_fixed_filtered.bed
134446 Dnase_non_enhancers_2000bp_extend_fixed_filtered.bed
```

```
> wc -l Dnase_enhancers_2000bp_extend_fixed.bed 
95033 Dnase_enhancers_2000bp_extend_fixed.bed
```

We lost ~40k non-enhancer peaks because they overalpped with enhancer regions. This is not surprising since Dnase peaks tend to be very small and there can be peaks next to each other where one overlaps with an ChromHMM enhancer region and the other does not:

![Dnase enhancer track](https://raw.githubusercontent.com/bwassie/enhancer_prediction/master/data/dnase_track_enhancer.png)

In the track above, The red regions are Dnase peaks and the blue regions are ChromHMM enhancer regions. It doesn't really make sense to call the long Dnase peak a non-enhancer peak since it's so close to enhancer called regions. We will just assume that by extending each peak to 2kb and labeling all the peaks in the image as enhancers, we will allow the CNN to pick up the enhancer signal. 

## Generating coverage files for uniform peaks 

Next, I will use bedtools coverage to generate basepair resolution coverage for the enhancer and non-enhancer regions:
```
>bedtools coverage -d -a Dnase_non_enhancers_2000bp_extend_fixed_filtered.bed -b E116-DNase.tagAlign > Dnase_non_enhancers_cov.txt
>bedtools coverage -d -a Dnase_enhancers_2000bp_extend_fixed.bed -b E116-DNase.tagAlign > Dnase_enhancers_cov.txt
```

Examining what the head of the coverage file looks like: 

```
chr1    121483777       121485777       PEAK1   10      .       1       0
chr1    121483777       121485777       PEAK1   10      .       2       0
chr1    121483777       121485777       PEAK1   10      .       3       0
chr1    121483777       121485777       PEAK1   10      .       4       0
chr1    121483777       121485777       PEAK1   10      .       5       0
```

and the middle of the coverage for the first peak:
```
chr1    121483777       121485777       PEAK1   10      .       991     4078
chr1    121483777       121485777       PEAK1   10      .       992     3699
chr1    121483777       121485777       PEAK1   10      .       993     3662
chr1    121483777       121485777       PEAK1   10      .       994     3337
chr1    121483777       121485777       PEAK1   10      .       995     3373
```

Towards the edge of the peaks, we have 0 coverage (which makes sense) and towards the middle, we get thousands of reads (which also makes sense). Good sanity check. 

Now cut the last column to get the coverage only:

```
>cut -f8 Dnase_enhancers_cov.txt > dnase_enhancers_cov_only.txt
>cut -f8 Dnase_non_enhancers_cov.txt > dnase_non-enhancers_cov_only.txt
```

Now I can use the coverage only files and `parse_coverage()` to create a numpy array for ML


In [1]:
import os
os.chdir("..\data")

In [2]:
from itertools import islice
import numpy as np

def parse_coverage(filename):
    arr = []
    n = 2000  # Or whatever chunk size you want
    with open(filename, 'rb') as f:
            for n_lines in iter(lambda: tuple(islice(f, n)), ()):
                    to_arr = np.asarray(n_lines,dtype=np.int)
                    arr.append(to_arr)


    arr = np.array(arr,np.int)
    return arr

In [3]:
import pickle, os, sys
import numpy as np

enh = parse_coverage("data\dnase_enhancers_cov_only.txt")
no_enh = parse_coverage("data\dnase_non-enhancers_cov_only.txt")
class_e = np.ones(enh.shape[0])
class_n = np.zeros(no_enh.shape[0])
classes = np.concatenate((class_e,class_n))
features = np.concatenate((enh,no_enh))
print("Number of enhancers peaks: {} \nNumber of non-enhancer peaks: {}".format(enh.shape[0],no_enh.shape[0]))
print("Naive accuracy (largest class): {}".format(no_enh.shape[0]/(no_enh.shape[0]+enh.shape[0])))

Number of enhancers peaks: 95033 
Number of non-enhancer peaks: 134446
Naive accuracy (largest class): 0.5858749602360129


Now that I have the coverage data in a neat numpy array, I can do some ML. As a baseline, our algorithm should perform better than the naive accuracy: ~59%

Before I create a model, I will split the data into stratified training and testing sets:

In [4]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
features2 = np.expand_dims(features, axis=2)
X_train, X_test, y_train, y_test = train_test_split(features2, classes, test_size=0.4,shuffle=True, random_state=42,stratify=classes)
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

Using TensorFlow backend.


## Training a 1D CNN 

I will use the same CNN architecture usd by Anshul Kundaje's lab (Max Pooling -> Convolution -> Max Pooling --> Convolution --> Global Max Pooling --> FC layer 1 --> FC layer 2)

In [5]:
#from: https://www.kaggle.com/alexanderlazarev/simple-keras-1d-cnn-features-split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, GlobalMaxPooling1D, MaxPooling1D


def base_model():
    model = Sequential()
    model.add(MaxPooling1D(input_shape=(2000, 1)))
    model.add(Conv1D(filters=3, kernel_size=50, input_shape=(2000, 1), activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(MaxPooling1D())
    model.add(Conv1D(filters=3, kernel_size=50, activation = 'relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4)) 
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
    #model.summary()
    return model

Now training a CNN on over 200k 1X2000 arrays will be very computationally intensive. Since I am running this on my desktop, I will instead randomly subsample 40k rows from the training set and 10k rows from the testing set. 

In [6]:
index=np.random.randint(0,X_train.shape[0],40000)
index2=np.random.randint(0,X_test.shape[0],10000)
X_train_sub = X_train[index]
y_train_sub = y_train[index]
X_test_sub = X_test[index2]
y_test_sub = y_test[index2]

Neural Nets generally have a ton of paramters. I can use sklearn's GridsearchCV to find the best parameters. In particular, neural nets are sensitive to batch size([1](https://towardsdatascience.com/a-walkthrough-of-convolutional-neural-network-7f474f91d7bd)). This grid search is inspired by [Machine Learning Mastery](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/).

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=base_model, verbose=0)
batch = [10, 60, 120]
epochs = [5,10,20]
param_grid = dict(batch_size=batch, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=5)
grid_result = grid.fit(X_train_sub, y_train_sub)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=5 .........................................
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_1 (MaxPooling1 (None, 1000, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 951, 3)            153       
_________________________________________________________________
dropout_1 (Dropout)          (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 475, 3)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 3)                 0         
___________

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.2min remaining:    0.0s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_3 (MaxPooling1 (None, 1000, 1)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 951, 3)            153       
_________________________________________________________________
dropout_4 (Dropout)          (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 475, 3)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 3)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               2048      
__________

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.4min remaining:    0.0s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_5 (MaxPooling1 (None, 1000, 1)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 951, 3)            153       
_________________________________________________________________
dropout_7 (Dropout)          (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 475, 3)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 3)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               2048      
__________

...

[CV]  batch_size=60, epochs=10, score=0.6355658888454726, total= 6.3min
[CV] batch_size=60, epochs=10 ........................................
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_29 (MaxPooling (None, 1000, 1)           0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 951, 3)            153       
_________________________________________________________________
dropout_43 (Dropout)         (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 475, 3)            0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_15 (Glo (None, 3)                 0         

[CV]  batch_size=120, epochs=5, score=0.6112194399399541, total= 3.1min
[CV] batch_size=120, epochs=5 ........................................
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_39 (MaxPooling (None, 1000, 1)           0         
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 951, 3)            153       
_________________________________________________________________
dropout_58 (Dropout)         (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 475, 3)            0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_20 (Glo (None, 3)                 0         

[CV]  batch_size=120, epochs=10, score=0.5665641629529591, total= 6.2min
[CV] batch_size=120, epochs=20 .......................................
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_49 (MaxPooling (None, 1000, 1)           0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 951, 3)            153       
_________________________________________________________________
dropout_73 (Dropout)         (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_50 (MaxPooling (None, 475, 3)            0         
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_25 (Glo (None, 3)                 0        

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 215.4min finished


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_55 (MaxPooling (None, 1000, 1)           0         
_________________________________________________________________
conv1d_55 (Conv1D)           (None, 951, 3)            153       
_________________________________________________________________
dropout_82 (Dropout)         (None, 951, 3)            0         
_________________________________________________________________
max_pooling1d_56 (MaxPooling (None, 475, 3)            0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 426, 3)            453       
_________________________________________________________________
global_max_pooling1d_28 (Glo (None, 3)                 0         
_________________________________________________________________
dense_82 (Dense)             (None, 512)               2048      
__________

A batch size of 60 and 20 epochs seems to be the best parameters. Batch of 120 also does well with 20 epochs. It seems that after a certain batch size, the accuracy is around the same. Earlier, we said the naive case accuracy is 59%. We get 6% impovement over the naive case. Chromputer achieves 68% accuracy for 8 different classes (with a naive accuracy of 42%) with Dnase alone. 

## Future directions

This was a nice proof of concept to show that CNNs can extract useful information from Dnase coverage. I was able to get a small improvement in accuracy over the naive case. It might be interesting to see if a less complicated algorithm can acheive similar results with less expensive computing; it took 215 minutes (!) for this CNN to run on my computer. Additionally, it might be interesting to see how adding other features (e.g. sequence information, ATACs-seq, H3K4me1 ChIP...etc) can affect the accuracy.  